<a href="https://colab.research.google.com/github/nandhukumar86/mykagglesubmissions/blob/master/whats_cooking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install kaggle
! pip install -q kaggle

# upload file to Colab
from google.colab import files
files.upload()

# create kaggle folder in root and copy the file.
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

# provide permissions
! chmod 600 ~/.kaggle/kaggle.json

# download competition files by providing competition name
! kaggle competitions download -c 'whats-cooking'

# unzip the files and folders
#! unzip train.csv.zip -d train
#! unzip test.csv.zip -d test


Saving kaggle.json to kaggle.json
  0% 0.00/426k [00:00<?, ?B/s]
100% 426k/426k [00:00<00:00, 63.6MB/s]
  0% 0.00/25.8k [00:00<?, ?B/s]
100% 25.8k/25.8k [00:00<00:00, 28.1MB/s]
  0% 0.00/1.76M [00:00<?, ?B/s]
100% 1.76M/1.76M [00:00<00:00, 119MB/s]


In [2]:
! unzip /content/train.json.zip -d train
! unzip /content/test.json.zip -d test

Archive:  /content/train.json.zip
  inflating: train/train.json        
   creating: train/__MACOSX/
  inflating: train/__MACOSX/._train.json  
Archive:  /content/test.json.zip
  inflating: test/test.json          
   creating: test/__MACOSX/
  inflating: test/__MACOSX/._test.json  


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline


In [4]:
test_raw = pd.read_json('/content/test/test.json')
train_raw = pd.read_json('/content/train/train.json')

In [5]:
train_raw.columns

Index(['id', 'cuisine', 'ingredients'], dtype='object')

In [6]:
train_raw.dtypes

id              int64
cuisine        object
ingredients    object
dtype: object

In [89]:
train_raw['ingredients'] = train_raw['ingredients'].astype('str')
test_raw['ingredients'] = test_raw['ingredients'].astype('str')

In [91]:
train_raw.head()

,id,cuisine,ingredients
0,10259,greek,"['romaine lettuce', 'black olives', 'grape tom..."
1,25693,southern_us,"['plain flour', 'ground pepper', 'salt', 'toma..."
2,20130,filipino,"['eggs', 'pepper', 'salt', 'mayonaise', 'cooki..."
3,22213,indian,"['water', 'vegetable oil', 'wheat', 'salt']"
4,13162,indian,"['black pepper', 'shallots', 'cornflour', 'cay..."


In [44]:
train_raw_ingredients = train_raw.ingredients

In [108]:
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()

# x = mlb.fit_transform(train_raw.ingredients)

from sklearn.feature_extraction.text import CountVectorizer
cV = CountVectorizer(analyzer = "word")
x = cV.fit_transform(train_raw.ingredients)

In [109]:
train_cols = cV.get_feature_names()

In [111]:
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [112]:
train_df_counts = pd.DataFrame(x.toarray(), columns = cV.get_feature_names())

In [119]:
train_df_counts.shape

(39774, 3010)

In [114]:

x = cV.transform(test_raw.ingredients)

In [115]:
test_df_counts = pd.DataFrame(x.toarray(), columns = cV.get_feature_names())

In [118]:
test_df_counts.shape

(9944, 3010)

In [120]:
X_train = train_df_counts
X_test = test_df_counts

In [121]:
print(X_train.shape)
print(X_test.shape)


(39774, 3010)
(9944, 3010)


In [122]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
labels = encoder.fit_transform(train_raw.cuisine)

In [125]:
y_train = labels

In [126]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

model = MultinomialNB() # Features - Discrete or Categorical or frequencies

model.fit(X_train, y_train)

y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)



In [127]:
print('Training Score: ', model.score(X_train, y_train))
# print('Testing Score: ', model.score(X_test, y_test))

# from sklearn.metrics import accuracy_score
# print('Accuracy: ',accuracy_score(y_test, y_test_predict))


Training Score:  0.7445567456127118


In [130]:
df = pd.concat([pd.DataFrame(test_raw.id),pd.DataFrame(encoder.inverse_transform(y_test_predict))], axis=1, ignore_index=True)
df.columns = ['id','cuisine']
df.to_csv('Submission.csv', index=False)
